In [1]:
import pandas as pd
from src import Utils, LSTM

/tmp/ipykernel_55544/519450756.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/nate/miniconda3/lib/python3.9/site-packages/spacy/util.py:918: UserWarning: [W094] Model 'tl_calamancy_md' (0.1.0) specifies an under-constrained spaCy version requirement: >=3.5.0. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.7.4,<3.8.0
  warnings.warn(warn_msg)
/home/nate/minicond

In [2]:
MODEL_FOLDER = 'model_lstm'

In [3]:
dataset = Utils.read_csv_file('datasets/datasetall.csv')
dataset

CSV file read successfully!


,text,label
0,Binay: Patuloy ang kahirapan dahil sa maling p...,0
1,SA GOBYERNONG TAPAT WELCOME SA BAGUO ANG LAHAT...,0
2,wait so ur telling me Let Leni Lead mo pero NY...,1
3,[USERNAME]wish this is just a nightmare that ...,0
4,doc willie ong and isko sabunutan po,0
...,...,...
28456,"Bisaya, Probinsyano/a, mostly Bisaya = katulong",1
28457,Amnesia. In my whole life wala pa ako nakasala...,1
28458,Kontrabida na ilang beses na tinalo at obvious...,1
28459,Yung antagonist laging kailangang sobrang sama...,1


In [4]:
Utils.seed_random_number_generators()

Random number generators seeded.


In [5]:
TEST_SIZE = 0.5

X_train, X_test, y_train, y_test = Utils.get_train_test_split(dataset, TEST_SIZE)

In [6]:
X_train

0        Iboboto ko talaga tong si binay eh nakikita ko...
1                       Dahil Pink WednesdaysLet Leni Lead
2        I don't know why but I feel irritated with the...
3        [USERNAME] AYY WAIT WALA PALA ANG MANOK KO??!!...
4        Worked as behavior analyst. Maraming parents n...
                               ...                        
14226                     Tangina talaga ng ad ni binay eh
14227    Something smells fishy... Philrem reeks of a m...
14228                    queen herself spoke let leni lead
14229    Luh Binay ranks 1 in the latest SWS survey. An...
14230                                Uunlad tayo kay Binay
Name: text, Length: 14231, dtype: object

In [7]:
y_train

0        0
1        0
2        1
3        0
4        0
        ..
14226    1
14227    1
14228    0
14229    1
14230    0
Name: label, Length: 14231, dtype: int64

In [8]:
X_test

0        Dahil ni Binaydi ko napanood ang ending ng Pan...
1        Pretty Little Liars ( starring Binay Family) N...
2        Napaka biased mo BOY ABUNDA! But you cant pin ...
3        [USERNAME] [USERNAME]and[USERNAME] wawafu ng k...
4        Pati mga mayor ng Home based ni VP Leni ninaka...
                               ...                        
14225     [USERNAME] and [USERNAME] troll spotted Pag g...
14226    Zaxezaxezaxe [USERNAME][USERNAME] STYLE NYO BU...
14227                          Sad walang paganito sa amin
14228    Oh my god sobrang nakakairita yung commercial ...
14229    Naga s Pink panalo Natin Para Sa Lahat Let Len...
Name: text, Length: 14230, dtype: object

In [9]:
y_test

0        0
1        1
2        1
3        0
4        1
        ..
14225    1
14226    1
14227    0
14228    1
14229    0
Name: label, Length: 14230, dtype: int64

In [10]:
epochs = [100, 200, 300]
learning_rate = [0.1, 0.01, 0.001, 0.0001, 0.00001]
batch_size = [32, 64, 128]
hidden_size = [100, 200, 300, 400]
num_layers = [1, 2, 3]

train_lstm = LSTM.LstmPipeline

In [11]:
train_lstm.set_params(
  # lstm__train_split=None,
  lstm__module__hidden_size=400,
  lstm__optimizer__lr=0.015,
  lstm__optimizer__weight_decay=0.00001,
  lstm__max_epochs=20,
  lstm__batch_size=32,
)

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=400,
))])

In [12]:
train_lstm

Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[uninitialized](
  module=<class 'src.LSTM.LstmModel'>,
  module__hidden_size=400,
))])

In [13]:
train_lstm.fit(X_train, y_train)

  0%|          | 0/445 [00:00<?, ?it/s]

  epoch    train_loss    valid_acc    valid_loss    cp     dur
-------  ------------  -----------  ------------  ----  ------
      1        0.7310       0.4949        0.7161     +  6.8042


  0%|          | 0/445 [00:00<?, ?it/s]

      2        0.7008       0.4935        0.7195        6.8710


  0%|          | 0/445 [00:00<?, ?it/s]

...

  0%|          | 0/445 [00:00<?, ?it/s]

     19        0.7157       0.4960        0.6941     +  6.7109


  0%|          | 0/445 [00:00<?, ?it/s]

     20        0.7205       0.5044        0.7309        6.7656


Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 400, num_layers=2, batch_first=True)
    (fc): Linear(in_features=400, out_features=2, bias=True)
  ),
))])

In [14]:
accuracy, recall, precision, f1 = Utils.get_prediction_results(
  X_test,
  y_test,
  train_lstm,
)

Accuracy: 0.49599437807449054
Recall: 0.00013941168269901018
Precision: 1.0
F1-score: 0.0002787844995818233


In [15]:
Utils.save_trained_model(train_lstm, f"{MODEL_FOLDER}/LSTM")

Ensemble model saved to Pipeline(steps=[('tokenizer', CalamancyTokenizer()),
                ('lstm',
                 <class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=LstmModel(
    (lstm): LSTM(200, 400, num_layers=2, batch_first=True)
    (fc): Linear(in_features=400, out_features=2, bias=True)
  ),
))]).pkl


In [16]:
history_data_frame = pd.DataFrame(
    train_lstm['lstm'].history
).set_index('epoch')
history_data_frame.to_csv(f'{MODEL_FOLDER}/lstm_history.csv')

In [17]:
history_data_frame

,batches,train_batch_count,valid_batch_count,dur,train_loss,train_loss_best,valid_loss,valid_loss_best,valid_acc,valid_acc_best,event_cp
epoch,,,,,,,,,,,
1,"[{'train_loss': 0.6879977583885193, 'train_bat...",356,89,6.804182,0.730988,True,0.716061,True,0.494907,True,True
2,"[{'train_loss': 0.6787959337234497, 'train_bat...",356,89,6.870956,0.700804,True,0.719503,False,0.493502,False,False
3,"[{'train_loss': 0.6946989297866821, 'train_bat...",356,89,6.849821,0.702227,False,0.721914,False,0.504391,True,False
4,"[{'train_loss': 0.7113974094390869, 'train_bat...",356,89,6.436294,0.703532,False,0.702041,True,0.497366,False,True
5,"[{'train_loss': 0.7546191811561584, 'train_bat...",356,89,6.781809,0.710268,False,0.705442,False,0.495961,False,False
6,"[{'train_loss': 0.6961996555328369, 'train_bat...",356,89,6.785497,0.708444,False,0.730459,False,0.489638,False,False
7,"[{'train_loss': 0.7673064470291138, 'train_bat...",356,89,6.615698,0.711280,False,0.715315,False,0.498068,False,False
8,"[{'train_loss': 0.7752690315246582, 'train_bat...",356,89,6.570146,0.715863,False,0.709945,False,0.494204,False,False
9,"[{'train_loss': 0.7529148459434509, 'train_bat...",356,89,6.603574,0.721839,False,0.700097,True,0.504039,False,True


In [18]:
metrics_data_frame = pd.DataFrame([{
  'accuracy': accuracy,
  'recall': recall,
  'precision': precision,
  'f1': f1,
}])
metrics_data_frame.to_csv(f'{MODEL_FOLDER}/lstm_metrics.csv')